In [7]:
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [8]:
img = cv2.imread('teste1.jpeg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [9]:
ret, thresh = cv2.threshold(gray, 175, 255, cv2.THRESH_BINARY)
#ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)
cv2.imshow('imagem2', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

In [11]:
area = []
from copy import copy
for i, ctr in enumerate(contours):
    x, y, w, h = cv2.boundingRect(ctr)
    area.append(w*h)
aux = copy(area)
aux.sort(reverse = True)


In [12]:
for i, ctr in enumerate(contours):
    x, y, w, h = cv2.boundingRect(ctr)

    if 0.25*aux[1] < area[i]:
        rect = cv2.rectangle(thresh, (x, y), (x + w, y + h), (0, 255, 0), 2) 
        cv2.imshow('rect', rect)
        cv2.imwrite(str(i)+".jpg",thresh[y:y+h,x:x+w])
        i=i+1

cv2.waitKey(0)
cv2.destroyAllWindows()